In [4]:
import hashlib

def hash_block(data, prev_hash):
    return hashlib.sha256((data + prev_hash).encode()).hexdigest()

# Genesis block (blok pertama)
blockchain = []
genesis_data = "Genesis Block"
genesis_hash = hash_block(genesis_data, "0")
blockchain.append((genesis_data, genesis_hash))

# Tambah blok berikutnya
data_2 = "Transaksi: Alice -> Bob 1 BTC"
hash_2 = hash_block(data_2, genesis_hash)
blockchain.append((data_2, hash_2))

# Cetak hasil
for i, (data, h) in enumerate(blockchain):
    print(f"Blok {i}: {data}")
    print(f"Hash : {h}\n")


Blok 0: Genesis Block
Hash : 8500b59bb5271135cd9bcbf0afd693028d76df3b9c7da58d412b13fc8a8f9394

Blok 1: Transaksi: Alice -> Bob 1 BTC
Hash : c89629827cfa6bd4178dc80cd6a7dc0181ff128f99fe8126b54870d1ab883a66



In [5]:
def mine_block(data, prev_hash, prefix="000"):
    nonce = 0
    while True:
        text = data + prev_hash + str(nonce)
        hash_val = hashlib.sha256(text.encode()).hexdigest()
        if hash_val.startswith(prefix):
            return nonce, hash_val
        nonce += 1

data_3 = "Transaksi: Budi -> Charlie 0.5 BTC"
nonce, mined_hash = mine_block(data_3, hash_2)
print(f"Nonce ditemukan: {nonce}")
print(f"Hash blok: {mined_hash}")


Nonce ditemukan: 4550
Hash blok: 000674d5791c3160f05196b292d1c8639dc5edbea90420873634c85001705192


In [6]:
def buat_transaksi(pengirim, penerima, jumlah):
    data = f"{pengirim}->{penerima}:{jumlah}"
    tx_hash = hashlib.sha256(data.encode()).hexdigest()
    return {"data": data, "hash": tx_hash}

tx1 = buat_transaksi("Alice", "Bob", 1)
print(tx1)


{'data': 'Alice->Bob:1', 'hash': '5f27d8b5bf1628a5f828e4ae177e6b52c4e62e7131df7993bc50ba51e7391e5c'}


In [7]:
transaksi = [
    buat_transaksi("Alice", "Bob", 1),
    buat_transaksi("Budi", "Charlie", 0.5)
]

block_data = "".join([tx["hash"] for tx in transaksi])
nonce, block_hash = mine_block(block_data, mined_hash)

print(f"Blok baru dengan {len(transaksi)} transaksi:")
print(f"Nonce: {nonce}\nHash: {block_hash}")


Blok baru dengan 2 transaksi:
Nonce: 2680
Hash: 00018ee0451a7913325c0a12853919d530de0e44903730f98415bd8c05bffec3


In [8]:
# Simulasi lengkap: "dari awal dibangun sampai transaksi & mining" — dicetak step-by-step.
# Program ini dibuat untuk dijalankan di Jupyter / Python REPL.
# - Bagian A: membangun blockchain sederhana (genesis)
# - Bagian B: membuat transaksi dan mempool
# - Bagian C: Proof-of-Work matematis sederhana (modulus) — **mencetak semua percobaan** (cepat)
# - Bagian D: Proof-of-Work menggunakan SHA-256 prefix (mirip nyata) — mencetak awal + periodic progress
# - Bagian E: memvalidasi blockchain dan mensimulasikan broadcast/consensus sederhana

import hashlib, time

def sha256(s: str) -> str:
    return hashlib.sha256(s.encode()).hexdigest()

class Transaction:
    def __init__(self, sender, recipient, amount):
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.data = f"{sender}->{recipient}:{amount}"
        self.tx_hash = sha256(self.data)
    def __repr__(self):
        return f"Tx({self.data}, hash={self.tx_hash[:16]}...)"

class Block:
    def __init__(self, index, prev_hash, transactions, timestamp=None):
        self.index = index
        self.prev_hash = prev_hash
        self.transactions = transactions  # list of Transaction
        self.timestamp = timestamp if timestamp is not None else time.time()
        self.nonce = None
        self.hash = None
    def tx_hash_concat(self):
        return "".join([tx.tx_hash for tx in self.transactions])
    def compute_hash(self, nonce):
        # Representasi string blok -> hash sha256
        block_string = f"{self.index}|{self.timestamp}|{self.prev_hash}|{self.tx_hash_concat()}|{nonce}"
        return sha256(block_string)
    def short(self):
        return {
            "index": self.index,
            "prev_hash": self.prev_hash[:16]+"...",
            "tx_count": len(self.transactions),
            "nonce": self.nonce,
            "hash": (self.hash[:16]+"..." if self.hash else None)
        }

# ---------- A. Membuat Genesis Block ----------
print("\n===== A. Membuat Genesis Block =====")
blockchain = []
genesis_txs = []  # biasanya genesis kosong atau special
genesis = Block(index=0, prev_hash="0"*64, transactions=genesis_txs, timestamp=1620000000.0)
# Untuk genesis kita tetapkan nonce/hashes sederhana
genesis.nonce = 0
genesis.hash = genesis.compute_hash(genesis.nonce)
blockchain.append(genesis)
print("Genesis block dibuat:")
print("  index:", genesis.index)
print("  prev_hash:", genesis.prev_hash[:16]+"...")
print("  hash:", genesis.hash)
print("  nonce:", genesis.nonce)

# ---------- B. Membuat transaksi & mempool ----------
print("\n===== B. Membuat transaksi & mempool =====")
mempool = []
# Contoh beberapa transaksi
mempool.append(Transaction("Alice", "Bob", 1.0))
mempool.append(Transaction("Budi", "Charlie", 0.5))
mempool.append(Transaction("Dewi", "Eka", 0.25))
mempool.append(Transaction("Alice", "Fajar", 0.1))

print("Transaksi dibuat dan dimasukkan ke mempool (lihat TxHash singkat):")
for i, tx in enumerate(mempool):
    print(f"  Tx#{i} -> {tx.data}  | tx_hash={tx.tx_hash}")

# ---------- C. Proof-of-Work sederhana (modulus) — semua proses dicetak ----------
# Ini ilustrasi matematika PoW yang mudah: penuhi syarat int(hash) % target_mod == 0
print("\n===== C. Proof-of-Work sederhana (modulus) — mencetak semua percobaan =====")
target_mod = 97  # target modulus: peluang ~1/97
candidate_block = Block(index=1, prev_hash=genesis.hash, transactions=mempool)
nonce = 0
start = time.time()

while True:
    bhash = candidate_block.compute_hash(nonce)
    modv = int(bhash, 16) % target_mod
    print(f"  mencoba nonce={nonce:5d}  hash={bhash}  int%{target_mod}={modv}")
    if modv == 0:
        candidate_block.nonce = nonce
        candidate_block.hash = bhash
        print("\n  >>> PoW sederhana SUCCESS!")
        print("    nonce valid:", nonce)
        print("    block hash :", bhash)
        break
    nonce += 1

elapsed = time.time() - start
print(f"  Selesai setelah {nonce+1} percobaan, waktu {elapsed:.4f} detik")

# Tambahkan ke blockchain
blockchain.append(candidate_block)
print("  Blok ditambahkan ke blockchain (index, hash singkat):", candidate_block.short())

# ---------- D. PoW SHA-256 (mirip nyata) — cetak awal + periodic progress ----------
print("\n===== D. Proof-of-Work menggunakan SHA-256 (prefix '000') =====")
# Buat transaksi baru untuk blok berikutnya (agar data berbeda)
mempool2 = [
    Transaction("Gilang", "Hana", 0.75),
    Transaction("Ivan", "Jenny", 0.125)
]
print("  Transaksi baru untuk blok selanjutnya:", mempool2)
candidate_block2 = Block(index=2, prev_hash=candidate_block.hash, transactions=mempool2)

prefix = "000"  # difficulty (contoh kecil). Di dunia nyata bisa > 10+ nol.
nonce = 0
start = time.time()
max_attempts = 5_000_000  # batas aman supaya tidak infinite loop di demo
printed = 0
found = False

while nonce < max_attempts:
    bhash = candidate_block2.compute_hash(nonce)
    # print detail untuk nonce kecil (lihat proses awal)
    if nonce < 200:
        print(f"  try {nonce:6d} -> hash={bhash}")
        printed += 1
    # print periodic progress untuk memberi gambaran tanpa membanjiri output
    elif nonce % 100000 == 0:
        print(f"  progress: nonce={nonce:,}  current_hash_prefix={bhash[:8]}...")
    if bhash.startswith(prefix):
        candidate_block2.nonce = nonce
        candidate_block2.hash = bhash
        found = True
        print("\n  >>> PoW SHA-256 SUCCESS!")
        print("    nonce valid:", nonce)
        print("    block hash :", bhash)
        break
    nonce += 1

elapsed = time.time() - start
if not found:
    print(f"\n  Gagal menemukan hash dengan prefix '{prefix}' dalam {max_attempts} percobaan (waktu {elapsed:.2f}s).")
else:
    print(f"  Ditemukan setelah {nonce+1} percobaan, waktu {elapsed:.4f}s")

# Tambahkan jika berhasil
if found:
    blockchain.append(candidate_block2)

# ---------- E. Validasi chain sederhana & consensus illustration ----------
print("\n===== E. Validasi blockchain & ilustrasi consensus sederhana =====")

def validate_chain(chain):
    print("  Memeriksa konsistensi chain...")
    for i in range(1, len(chain)):
        prev = chain[i-1]
        curr = chain[i]
        recomputed = curr.compute_hash(curr.nonce)
        print(f"    memeriksa blok index={curr.index}: prev_hash_stored={curr.prev_hash[:16]}..., recomputed_hash={recomputed[:16]}..., stored_hash={curr.hash[:16]}...")
        if curr.prev_hash != prev.hash:
            return False, f"prev_hash mismatch di blok {curr.index}"
        if recomputed != curr.hash:
            return False, f"hash mismatch (data diubah?) di blok {curr.index}"
    return True, "valid"

valid, msg = validate_chain(blockchain)
print("  Hasil validasi:", valid, "|", msg)

print("\nRingkasan Blockchain (setiap blok singkat):")
for b in blockchain:
    s = b.short()
    print(f"  Blok {s['index']}: prev={s['prev_hash']} txs={s['tx_count']} nonce={s['nonce']} hash={s['hash']}")

# Simulasi consensus (3 node): node A,B,C mulai dengan genesis kemudian node B punya rantai terpanjang -> node A,C adopsi
print("\nSimulasi sederhana consensus (longest chain rule):")
node_A = blockchain[:1]  # hanya genesis
node_B = blockchain[:]    # semua blok (A ikut kenaikan)
node_C = blockchain[:1]   # hanya genesis

print("  awal: len(A)=", len(node_A), "len(B)=", len(node_B), "len(C)=", len(node_C))
# Node A dan C minta update dari B -> adopt longest chain
if len(node_B) > len(node_A):
    node_A = [Block(b.index, b.prev_hash, b.transactions, b.timestamp) for b in node_B]  # simple copy
if len(node_B) > len(node_C):
    node_C = [Block(b.index, b.prev_hash, b.transactions, b.timestamp) for b in node_B]

print("  setelah sinkronisasi: len(A)=", len(node_A), "len(B)=", len(node_B), "len(C)=", len(node_C))
print("\n===== Selesai simulasi =====")





===== A. Membuat Genesis Block =====
Genesis block dibuat:
  index: 0
  prev_hash: 0000000000000000...
  hash: ceb1681ed0bddd9dd7e9d2c333a79d64ed6c10f0d6677a0220aae919dbc3b3ce
  nonce: 0

===== B. Membuat transaksi & mempool =====
Transaksi dibuat dan dimasukkan ke mempool (lihat TxHash singkat):
  Tx#0 -> Alice->Bob:1.0  | tx_hash=a282f87f0b9c393d747d23f57847608339b978537b5e0a0fa7ebebd51029bf05
  Tx#1 -> Budi->Charlie:0.5  | tx_hash=5bc2ae8f51cdec356a753f34c14993b54872063e372e6cefb9c38718e548ebbd
  Tx#2 -> Dewi->Eka:0.25  | tx_hash=972d2d6114effea66f1841eb0ec86de33ce8fcf9789a2fae8354127f39a16a70
  Tx#3 -> Alice->Fajar:0.1  | tx_hash=df35e96785d754f6379a2cb378407c91714c78e515519f1603dee4d30ad365b7

===== C. Proof-of-Work sederhana (modulus) — mencetak semua percobaan =====
  mencoba nonce=    0  hash=c490cbcb9ead7f7ecb572ada3d0735f633eedda079d545c7d59e70578db50c97  int%97=44
  mencoba nonce=    1  hash=51b2be962ff667695b0414d8ce8eed38ed69b26c2dbf858116559e2ce98ea6a2  int%97=48
  menco